In [0]:
# dbutils.fs.mkdirs("/Volumes/recargapay/recargapay/vol_rp/bronze")

In [0]:
# dbutils.fs.mkdirs("/Volumes/recargapay/recargapay/vol_rp/silver")

In [0]:
# dbutils.fs.mkdirs("/Volumes/recargapay/recargapay/vol_rp/gold")

In [0]:
df_transactions = spark.read.format('parquet').option('header', 'true').option('inferSchema', 'true').load("/Volumes/recargapay/recargapay/vol_rp/bronze/*.parquet").orderBy("account_id", "event_time")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, sum as _sum, to_date, min as _min, max as _max,
    expr
)

# Start Spark session
spark = SparkSession.builder.appName("AccountTransactions").getOrCreate()

# 1. Format date and aggregate transaction amounts per day
df_daily_transactions = (
    df_transactions.withColumn("date", to_date("event_time"))
      .groupBy("account_id", "user_id", "date")
      .agg(_sum("amount").alias("daily_amount"))
)

# 2. Get date range (start and end date) per account
df_range = df_daily_transactions.groupBy("account_id").agg(
    _min("date").alias("start_date"),
    _max("date").alias("end_date")
)

# 3. Create a sequence of dates for each account
df_all_dates = df_range.select(
    col("account_id"),
    expr("sequence(start_date, end_date)").alias("all_dates")
).withColumn("date", expr("explode(all_dates)")).drop("all_dates")

# 4. Join with unique combinations of account and user
df_users = df_daily_transactions.select("account_id", "user_id").distinct()

df_complete = (
    df_all_dates
    .join(df_users, on="account_id")
    .join(df_daily_transactions, on=["account_id", "user_id", "date"], how="left")
    .fillna(0, subset=["daily_amount"])
)

# 5. Final result: df_complete
df_complete.orderBy("account_id", "date").display()


In [0]:
df_complete.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("date") \
    .save("/Volumes/recargapay/recargapay/vol_rp/silver/account_daily_amount")

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, sum as _sum, when, round, col

# 1. Create a window partitioned by account and ordered by date
account_window = Window.partitionBy("account_id").orderBy("date")

# 2. Cumulative balance per account (daily_balance)
df_complete = df_complete.withColumn(
    "daily_balance",
    round(_sum("daily_amount").over(account_window), 2)
)

# 3. Previous day's balance
df_complete = df_complete.withColumn(
    "prev_balance",
    lag("daily_balance", 1).over(account_window)
)

# 4. 1% interest on balances greater than 100
df_complete = df_complete.withColumn(
    "interest_above_100",
    when(col("daily_balance") > 100, round(col("daily_balance") * 0.01, 2)).otherwise(0.0)
)

# 5. 0.5% interest on previous balance (only if there was no transaction)
df_complete = df_complete.withColumn(
    "inactive_interest",
    when(col("daily_amount") == 0, round(col("prev_balance") * 0.005, 2)).otherwise(0.0)
)

# 6. Total including interests
df_complete = df_complete.withColumn(
    "total_with_interest",
    round(col("daily_balance") + col("interest_above_100") + col("inactive_interest"), 2)
)

# Display result
df_complete.select(
    "account_id", "user_id", "date", "daily_amount",
    "daily_balance", "prev_balance",
    "interest_above_100", "inactive_interest", "total_with_interest"
).orderBy("account_id", "date").display()


In [0]:
df_complete.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("date") \
    .save("/Volumes/recargapay/recargapay/vol_rp/silver/account_daily_interest_amount")

In [0]:
from pyspark.sql.functions import expr

# Assume the following DataFrame was loaded as the control table
df_rates = spark.createDataFrame([
    ("2024-01-01", "2024-12-31", 0.01, 0.005),
    ("2025-01-01", "2025-12-31", 0.008, 0.004),
], ["start_validity", "end_validity", "rate_above_100", "rate_inactive"]) \
    .withColumn("start_validity", to_date("start_validity")) \
    .withColumn("end_validity", to_date("end_validity"))

# Join with the transactions per date (df_complete)
df_final = df_complete.join(
    df_rates,
    (df_complete["date"] >= df_rates["start_validity"]) & 
    (df_complete["date"] <= df_rates["end_validity"]),
    how="left"
)

# Apply interests based on dynamic rates
df_final = df_final.withColumn(
    "interest_above_100",
    when(col("daily_balance") > 100, round(col("daily_balance") * col("rate_above_100"), 2)).otherwise(0.0)
)

df_final = df_final.withColumn(
    "inactive_interest",
    when(col("daily_amount") == 0, round(col("prev_balance") * col("rate_inactive"), 2)).otherwise(0.0)
)

df_final = df_final.withColumn(
    "total_with_interest",
    round(col("daily_balance") + col("interest_above_100") + col("inactive_interest"), 2)
)

df_final = df_final.withColumn(
    "year_month",
    expr("date_format(date, 'yyyyMM')")
)

# Display final result
df_final.select(
    "account_id", "user_id", "date", "daily_amount",
    "daily_balance", "prev_balance",
    "rate_above_100", "rate_inactive",
    "interest_above_100", "inactive_interest", "total_with_interest","year_month"
).orderBy("account_id", "date").display()


In [0]:
df_final.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year_month") \
    .save("/Volumes/recargapay/recargapay/vol_rp/gold/account_daily_summary")